<a href="https://colab.research.google.com/github/CristhianSeverino/customer-churn-and-contextual-chat/blob/staging/DSMarketWiFi_DataCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customer Churn and Contextual Chat C4**

> Este **Notebook** es una creación de: **Cristhian Calle Severino**.

---

* **GitHub**: https://github.com/CristhianSeverino
* **LinkedIn**: https://www.linkedin.com/in/cristhianandrescalleseverino/

---

Este proyecto **emula un Data Product de Machine Learning (ML) con despliegue en AWS**. Se inspira en la arquitectura **Arana**.

### **Incluye:**

* **Creación de Datos Sintéticos**: Estos datos simulan usuarios de **MallPlaza**, un nombre genérico para un supuesto ficticio, dado que grandes centros comerciales homónimos existen en ciudades de América (como Santiago de Chile y Manizales, donde resido). Esta creación se basa en un supuesto de **Ingeniería y Arquitectura de Datos** que se detalla en el **Road Map**.

---

## **Módulos del Proyecto**

El proyecto está estructurado en los siguientes **Módulos**:

> * **Módulo 1 - Creación de Datos Sintéticos**: Emula un proceso **ELT** (Extracción, Carga y Transformación) de ingesta, carga y posterior transformación de datos. El resultado es un *dataset* limpio, listo para el entrenamiento de modelos de ML.
> * **Módulo 2 - EDA (*Exploratory Data Analysis*)**: Análisis de datos exploratorio para la identificación de *insights* previos al ML. Incluye la búsqueda de características de enriquecimiento y el desarrollo de **Feature Engineering**.
> * **Módulo 3 - Entrenamiento y Evaluación del Algoritmo**: Desarrollo del modelo, métricas de ajuste, evaluación de precisión y **Fine Tuning**.
> * **Módulo 4 - Despliegue en AWS**: Implementación y prueba del despliegue en la nube.

---

## **Prerrequisitos y Recomendaciones**

**Nota**: Para este proyecto se emplea **AWS como *cloud platform***.

Es **indispensable** poseer conocimientos en las siguientes tecnologías para comprender a cabalidad los aspectos técnicos:

* **Lenguajes y Bases de Datos**: Python, SQL.
* **Big Data y ML**: PySpark, Keras, Scikit-learn, PyTorch.
* **Cloud y Herramientas MLOps**: AWS.
* **IA y Desarrollo de Aplicaciones**: LangChain, RAG, Streamlit, Gradio.

Para una comprensión general del proyecto, **recomiendo encarecidamente la lectura del Road Map** y una revisión general de la documentación del código. 🦾

> ¡Que disfrutes explorando este proyecto! Exitosa semana. 🐱‍🏍

# **Instalacción E Importación de Librerias**📚

In [6]:
!pip install boto3
!pip install awscli
print("="*150)
print(" "*50+"Librerias AWS instaladas")
print("="*150)

                                                  Librerias AWS instaladas


In [5]:
!pip install faker
print("="*150)
print(" "*50+"Librerias instaladas")
print("="*150)


                                                  Librerias instaladas


In [34]:
import os
from google.colab import userdata
from faker import Faker
import pandas as pd
from datetime import datetime
import random
import numpy as np
import boto3
print("="*150)
print(" "*50+"Librerias Importadas con Exito ☕")
print("="*150)

                                                  Librerias Importadas con Exito ☕


# **Credenciales**🎫

**Ojo Actualizar Credenciales para el rol del Proyecto. Segun Data gobernance**👀

In [39]:
aws_access_key_id = userdata.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = userdata.get('AWS_SECRET_ACCESS_KEY')
region_name=userdata.get('AWS_REGION')
bucket_name=userdata.get('s3_mallflow')
git_path= userdata.get('GIT_PATH')
s3 = boto3.client('s3')
print("="*150)
print(" "*50+"Credenciales Cargadas")
print("="*150)

                                                  Credenciales Cargadas


# **Generación de Data Set Sintetico**🤖

In [26]:
def generar_dataset_churn_wifi(n_muestras=1000, output_file='dataset_churn_mall_plaza.csv'):
    """Genera un Data Set Sintetico. este data set emula ingesta via api o Bukect. como data waehouse o data lake.
    Columnas Generadas:
    -customerId: ide unico (con Faker para mayor realismo)
    -name:  Nombre ficticio (con Faker para mayor realismo)
    -gender: genero (male/Female/Other)
    -age: Edad (18 - 80 años)
    -visits: Numero de Visitas al cenntro comercial/Wifi en el ultimo mes(1-30)
    -avg_session_time: Tiempo promedio por sesion en minutos (5-120)
    -complaints: Numero de quejas (0-10)
    -location: Area del centro Comercial: e.g. mesanine, Area de Comidas 1, area de comidas 2, cine, otros
    -divice_type: ipo de dispositivo (Smartphone/Laptop/Tablet
    -churn: Etiqueta de abadono (Yes/No), con probabilidad basada en quejas, visitas bajas y tiempo bajo.

    Args:
        n_muestras (int): Numero de muestras a generar.
        ouput_file (str): ruta/nombre del archivo csv donde se guardara el dataset.
    Returns
        pd.DataFrame: Dataset generado.

    """
    fake = Faker('es_CL') # Faker configurado para chile
    np.random.seed(42)
    random.seed(42)

    data = {
        'customerId': [fake.uuid4() for _ in range(n_muestras)],
        'name': [fake.name() for _ in range(n_muestras)],
        'gender': np.random.choice(['Male','Female','Other'], n_muestras, p=[0.485,0.51,0.005]),
        'age': np.random.randint(18,81,n_muestras),
        'visits': np.random.randint(1,31,n_muestras),
        'avg_session_time': np.random.uniform(5, 120, n_muestras).round(2),
        'complaints': np.random.randint(0,11,n_muestras),
        'location': [fake.random_element(elements=['Mall Central', 'Mega Tieda', 'Cinemas', 'Electronica','Mall Comidas 1', 'Mall Comidas 2', 'Gimnasios', 'Arcades']) for _ in range(n_muestras)],
        'device_type': np.random.choice(['Smartphone','Laptop', 'Table'], n_muestras, p=[0.85, 0.08, 0.07]),

    }
    df = pd.DataFrame(data)

    # Generar churn: Probabilidad alta si quejas >5, visitas < 5 o tiempo promedio <20
    prob_churn = 0.15 + (df['complaints'] / 10) * 0.4 + (1 - df['visits']/30) * 0.3 + (1 - df['avg_session_time']/120)*0.2
    prob_churn = np.clip(prob_churn, 0, 1)
    df['churn'] = np.where(np.random.rand(n_muestras) < prob_churn, 'Yes', 'No')


    #Guardar CSV
    df.to_csv(output_file, index=False)
    print(f"Dataset Generado 🧮 y guardado en '{output_file}' con {n_muestras} muestras")

    return df



In [27]:
df = generar_dataset_churn_wifi(n_muestras=100000, output_file='churn_abril_mallplaza.csv' )

print(df.head())


Dataset Generado 🧮 y guardado en 'churn_abril_mallplaza.csv' con 100000 muestras
                             customerId                           name  \
0  fdea3467-f965-470a-885f-f9f5d0e3cec7             Simón Cornejo Lara   
1  42c8ead5-1211-437d-83db-9c866cc95583           Cristian Silva López   
2  51f9aa89-32ab-4fd8-9af5-145684a6787d  Sebastián Juan Cortés Montero   
3  ddf803b9-5b97-470e-ac7c-389d7a146c3c     Pablo Mario Valdivia Pérez   
4  a45289dd-a37c-4913-b59b-281862d36ee7       Mónica Ana Parra Aravena   

   gender  age  visits  avg_session_time  complaints        location  \
0    Male   51       3             32.57           0  Mall Comidas 2   
1  Female   25      13             64.52           8     Electronica   
2  Female   79      27             84.52           2       Gimnasios   
3  Female   32      28             65.70           1      Mega Tieda   
4    Male   74      20             27.03           8  Mall Comidas 2   

  device_type churn  
0  Smartphone   Yes

# **Igesta de Datos en Bucket de AWS** 💾

In [36]:
# rear los archivos de configuración de AWS
# Directorio y Archivo de Credenciales
aws_dir = '/root/.aws'
os.makedirs(aws_dir, exist_ok=True)
with open(os.path.join(aws_dir, 'credentials'), 'w') as f:
    f.write(f'[default]\n')
    f.write(f'aws_access_key_id = {aws_access_key_id}\n')
    f.write(f'aws_secret_access_key = {aws_secret_access_key}\n')
# Write 'config'
with open(os.path.join(aws_dir, 'config'), 'w') as f:
    f.write(f'[default]\n')
    f.write(f'region = {region_name}\n')
print("Archivos de cofiguracion de AWs creados Exitosamente ☕")


Archivos de cofiguracion de AWs creados Exitosamente ☕


In [37]:
def upload_to_s3(local_file,s3_path):
    s3.upload_file(local_file, bucket_name, s3_path)
    print(f"Subido {local_file} a s3://{bucket_name}/{s3_path}")
upload_to_s3('churn_abril_mallplaza.csv', 'warehouse-ml/churn_abril_mallplaza/churn_abril_mallplaza.csv')

Subido churn_abril_mallplaza.csv a s3://mall-flow/warehouse-ml/churn_abril_mallplaza/churn_abril_mallplaza.csv


# ***Verifica la Crreacion de los CSV en el icono de carpeta.costado izquiero de colab***

> **Diviertete Explorando este Proyecto**

Si te fue de ayuda este proyecto, pasa por mi linkedin y cuentame como te ayudo ☕
